this is the inference from HDFS to HDFS in parquet format file

In [2]:
from src.dataset_utils import Dataset
from src.model import TFModel
from datetime import datetime
from config.config import INFERENCE_CONFIG
from pyspark.sql import SparkSession

2023-03-17 10:11:43.730477: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Prepare Dataset

In [6]:
#create spark 
spark = SparkSession.builder \
    .master("local[1]") \
    .appName("SparkByExamples.com") \
    .getOrCreate()
#preprocess before inference
RAW_FOLDER = INFERENCE_CONFIG['RAW_FOLDER']
RAW_FOLDER = f'{RAW_FOLDER}/*.parquet'
df = spark.read.parquet(RAW_FOLDER)
pd_df = df.toPandas()

In [7]:
pd_df = pd_df.rename(columns={
    '_1' : 'video_id',
    '_2' : 'description',
    '_3' : 'published_at',
    '_4' : 'channel_name',
    '_5' : 'video_title',
    '_6' : 'figure',
    '_7' : 'keyword'
})
pd_df = pd_df.drop_duplicates(subset='video_id')
pd_df = pd_df.dropna(subset=['description'])

In [6]:
pd_df.to_csv(INFERENCE_CONFIG['RAW_CSV_FILE'], index=False)

# Inference

In [4]:
#model inference
train = False
#prepare the inference data
print('now creating dataset'.center(60, '-'))
data = Dataset(train)
data.prepare_inference_data()

--------------------now creating dataset--------------------


In [5]:
model = TFModel()
y = model.inference(data.tfdf)

66/66 [==============================] - 1s 14ms/step


In [17]:
# datetime object containing current date and time
now = datetime.now()
# dd/mm/YY H:M:S
dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
data.df['result'] = y
data.df['inference_date'] = dt_string

In [ ]:
#Create PySpark DataFrame from Pandas
sparkDF=spark.createDataFrame(data.df) 
sparkDF.write.save(INFERENCE_CONFIG['INFERENCE_FOLDER'], format='parquet', mode='append')

In [12]:
#test hasil loading inference data
inference_folder = INFERENCE_CONFIG['INFERENCE_FOLDER']
load_df = spark.read.parquet(f'{inference_folder}/*.parquet')
load_df.show()

root
 |-- video_id: string (nullable = true)
 |-- description: string (nullable = true)
 |-- published_at: string (nullable = true)
 |-- channel_name: string (nullable = true)
 |-- video_title: string (nullable = true)
 |-- sentiment: long (nullable = true)

+-----------+--------------------+--------------------+--------------------+--------------------+---------+
|   video_id|         description|        published_at|        channel_name|         video_title|sentiment|
+-----------+--------------------+--------------------+--------------------+--------------------+---------+
|poZtdyC24P4|Tutorial belajar ...|2023-02-24T23:45:01Z|         Dea Afrizal|Belajar Dasar Pem...|        0|
|rfscVS0vtbw|This course will ...|2018-07-11T18:00:42Z|    freeCodeCamp.org|Learn Python - Fu...|        1|
|xETkm9H6aaY|Yuk Belajar Pytho...|2023-02-13T10:00:36Z|       Kelas Terbuka|Belajar Python [D...|        2|
|_uQrJ0TkZlc|Python tutorial -...|2019-02-18T15:00:08Z|Programming with ...|Python Tutorial -